In [37]:
import pandas as pd
filename = "../user/" + "prathamthepro@gmail.com" + "/AIFiles/" + "10xdev.ai" + ".csv"
fs = pd.read_csv(filename)

for index in range(0,71):
    display(fs['summary'][index])
    display(fs['role'][index])
    print("\n-----\n")

'This file contains the Flask application that serves as the backend for the web application. It imports various modules and functions from other files in the codebase, including Ask_AI, train_AI, IgnoreAI, UserLogger, and others. The file defines various routes for handling API requests, such as getting project information, training the AI, syncing files, getting data from the AI, and managing user authentication and authorization. The file also sets up a session for each user and creates a UserLogger instance for each user to log their interactions with the AI. Finally, the file runs the Flask application on port 8000.'

' Flask backend that serves as the API for the web application. It handles API requests, user authentication, and authorization. It also sets up user sessions and logs user interactions with the AI.\n'


-----



"This file contains a function called `syncAI` that is responsible for syncing the user's local files with the AI's knowledge base. It imports various utility functions and modules such as `pandas`, `subprocess`, `os`, `difflib`, and `time`. The function first pulls the latest changes from the user's Git repository and then checks for any changes in the user's local files. If a file has changed, it summarizes the changes using the `summarize` function and updates the AI's knowledge base with the new summary and embedding. If there are any new files, it adds them to the knowledge base and summarizes them as well. Finally, it saves the updated knowledge base to a CSV file and creates a clone of the user's local files for future comparisons."

" Contains a function called `syncAI` that syncs the user's local files with the AI's knowledge base. It pulls the latest changes from the user's Git repository, summarizes the changes, and updates the AI's knowledge base. It also saves the updated knowledge base to a CSV file and creates a clone of the user's local files.\n"

...

' Handles Mixpanel tracking events. It creates a Mixpanel client object and defines a function for tracking events using the client.\n'


-----



'The file AskAI.py contains several functions related to searching and filtering files based on user prompts. \n\nThe function `process_file_contents_with_langchain` takes in the contents of a file, a user prompt, and a maximum number of tokens. It applies stemming to the file contents, creates a TF-IDF matrix, and calculates the cosine similarity between the user prompt and each chunk of the file. It then selects the chunks with the highest similarity and returns them as a joined string.\n\nThe function `max_cosine_sim` calculates the maximum cosine similarity between a list of embeddings and a prompt embedding.\n\nThe function `filter_functions` filters a list of file paths based on a response string. It checks if each file path is mentioned in the response string using regular expressions and returns the filtered list.\n\nThe function `search_functions` takes in a code query, an email, a user logger, a scope, and a history flag. It filters the file paths based on the scope, calculat

' Contains functions related to searching and filtering files based on user prompts. It includes functions for processing file contents, filtering file paths, and searching for relevant files.\n'


-----



In [34]:
display(fs[pd.isnull(fs["role"])]['file_path'])

52                  utilities/rates.py
53    10xdev/src/UiComponents/alert.js
54            10xdev/src/NewWelcome.js
55           10xdev/src/LeftWelcome.js
56         utilities/notebook_utils.py
66         10xdev/src/Branch/Branch.js
Name: file_path, dtype: object